In [1]:
import datetime
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示
from zipfile import ZipFile
import numpy as np
import pandas as pd
import datetime
import gc


In [6]:
train = pd.read_csv("H:\\pythonchengx_u\\Ecomme AI\\B\\ECommAI_ubp_round1_train_2", delimiter='\t', header=None, names= ["user_id", "item_id", "behavior_type", "date"]) 

# train = train[train['date'] >= 20190616]
# gc.collect()
# 
# items = pd.read_csv("H:\\pythonchengx_u\\Ecomme AI\\B\\ECommAI_ubp_round1_item_feature_2", delimiter='\t', header=None, names= ["item_id", "class1_id", "class_id", "brand_id", "price"])
# items = pd.DataFrame(items, columns = ["item_id", "class_id"])
# features = features.merge(items[["item_id", "class_id"]], on = ['item_id'], how='inner')
# gc.collect()


In [7]:
train.head(2)

,user_id,item_id,behavior_type,date
0,1076667570,340804045,clk,20190613
1,1076667570,340804045,buy,20190613


In [17]:
test = pd.read_csv("H:\\pythonchengx_u\\Ecomme AI\\B\\ECommAI_ubp_round1_test_2", delimiter='\t', header=None, names= ["user_id", "item_id", "behavior_type", "date"]) 


In [4]:
test.head(2)

,user_id,item_id,behavior_type,date
0,100000432,NaN,NaN,NaN
1,100002167,NaN,NaN,NaN


In [8]:
train=train[train['behavior_type']=='buy']



In [11]:
train=train.drop(['behavior_type'],axis=1)
train=train.drop(['date'],axis=1)


In [14]:
train.head()


,user_id,item_id
1,1076667570,340804045
5,1076667570,273994489
11,1076667570,142049606
18,1076667570,1224612971
19,1076667570,525100650


In [18]:

test=test.drop(['behavior_type','date'],axis=1)

In [19]:
test.head()



,user_id,item_id
0,100000432,NaN
1,100002167,NaN
2,1000023017,NaN
3,1000030431,NaN
4,100003050,NaN


In [20]:
test=test.drop(['item_id'],axis=1)
test = train.merge(test, on = ['user_id'], how='inner')


In [21]:
test.head()

,user_id,item_id
0,1052625010,1229359742
1,1052625010,1148830908
2,1052625010,1140429766
3,1052625010,839302409
4,1052625010,1149498035


In [22]:
test_item=test
test_item.to_csv("H:\\pythonchengx_u\\Ecomme AI\\Date\\test_item",index=False)

In [25]:
train.to_csv("H:\\pythonchengx_u\\Ecomme AI\\Date\\train_item",index=False)


In [37]:
test=test_item
del test_item
gc.collect()

412

In [2]:
import sys
import random
import math
import os
from operator import itemgetter
from collections import defaultdict

random.seed(44)

class ItemBasedCF(object):
    ''' TopN recommendation - Item Based Collaborative Filtering '''

    def __init__(self):
        self.trainset = {}
        self.testset = {}

        self.n_sim_item = 100
        self.n_rec_item = 30

        self.item_sim_mat = {}
        self.item_popular = {}
        self.item_count = 0

        print('Similar item number = %d' % self.n_sim_item)
        print('Recommended item number = %d' % self.n_rec_item)

    def generate_dataset(self, train,test=None, pivot=0.7):
        ''' load rating data and split it to training set and test set '''
        trainset_len = 0
        testset_len = 0
        if test is None:# 随机分配验证机实验 
            print('算法尝试！')
            for line in train.iterrows():
                user, item, rating = line[1][0],line[1][1],1
                # split the data by pivot
                if random.random() < pivot:
                    self.trainset.setdefault(user, {})
                    self.trainset[user][item] = int(rating)
                    trainset_len += 1
                else:
                    self.testset.setdefault(user, {})
                    self.testset[user][item] = int(rating)
                    testset_len += 1
        else:#真正预测
            print('预测尝试！')
            for line in train.iterrows():
                user, item, rating = line[1][0],line[1][1],1
                # split the data by pivot
                self.trainset.setdefault(user, {})
                self.trainset[user][item] = int(rating)
                trainset_len += 1
            del user,item,rating
            for line in test.iterrows():
                user, item, rating = line[1][0],line[1][1],1
                self.testset.setdefault(user, {})
                self.testset[user][item] = int(rating)
                testset_len += 1
            
        print ('split training set and test set succ')
        print ('train set = %s' % trainset_len)
        print ('test set = %s' % testset_len)

    def calc_item_sim(self):
        ''' calculate item similarity matrix '''
        print('counting items number and popularity...')

        for user, items in self.trainset.items():
            for item in items:
                # count item popularity
                if item not in self.item_popular:
                    self.item_popular[item] = 0
                self.item_popular[item] += 1

        print('count items number and popularity succ')

        # save the total number of items
        self.item_count = len(self.item_popular)
        print('total item number = %d' % self.item_count)

        # count co-rated users between items
        itemsim_mat = self.item_sim_mat
        print('building co-rated users matrix...')

        for user, items in self.trainset.items():
            for m1 in items:
                itemsim_mat.setdefault(m1, defaultdict(int))
                for m2 in items:
                    if m1 == m2:
                        continue
                    itemsim_mat[m1][m2] += 1

        print('build co-rated users matrix succ')

        # calculate similarity matrix
        print('calculating item similarity matrix...')
        simfactor_count = 0
        PRINT_STEP = 2000000

        for m1, related_items in itemsim_mat.items():
            for m2, count in related_items.items():
                itemsim_mat[m1][m2] = count / math.sqrt(
                    self.item_popular[m1] * self.item_popular[m2])
                simfactor_count += 1
                if simfactor_count % PRINT_STEP == 0:
                    print('calculating item similarity factor(%d)' %
                          simfactor_count)

        print('calculate item similarity matrix(similarity factor) succ')
        print('Total similarity factor number = %d' % simfactor_count)

    def recommend(self, user,predict=False):
        ''' Find K similar items and recommend N items. '''
        K = self.n_sim_item
        N = self.n_rec_item
        rank = {}
        if not predict:
            watched_items = self.trainset[user]
        else:
            watched_items = self.testset[user]

        for item, rating in watched_items.items():
            if itemcf.item_sim_mat.get(item,None) is None:
                continue
            for related_item, similarity_factor in sorted(self.item_sim_mat[item].items(),
                                                           key=itemgetter(1), reverse=True)[:K]:
                if related_item in watched_items:
                    continue
                rank.setdefault(related_item, 0)
                rank[related_item] += similarity_factor * rating
        # return the N best items
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[:N]

    def evaluate(self):
        ''' print evaluation result: precision, recall, coverage and popularity '''
        print('Evaluation start...')

        N = self.n_rec_item
        #  varables for precision and recall
        hit = 0
        rec_count = 0
        test_count = 0
        # varables for coverage
        all_rec_items = set()
        # varables for popularity
        popular_sum = 0

        for i, user in enumerate(self.trainset):
            if i % 500 == 0:
                print ('recommended for %d users' % i)
            test_items = self.testset.get(user, {})
            rec_items = self.recommend(user)
            for item, _ in rec_items:
                if item in test_items:
                    hit += 1
                all_rec_items.add(item)
                popular_sum += math.log(1 + self.item_popular[item])
            rec_count += N
            test_count += len(test_items)

        precision = hit / (1.0 * rec_count)
        recall = hit / (1.0 * test_count)
        coverage = len(all_rec_items) / (1.0 * self.item_count)
        popularity = popular_sum / (1.0 * rec_count)

        print ('precision=%.4f\trecall=%.4f\tcoverage=%.4f\tpopularity=%.4f' %
               (precision, recall, coverage, popularity))
    def predict(self):
        print('Predict start...')

        N = self.n_rec_item
        #  varables for precision and recall
        hit = 0
        rec_count = 0
        test_count = 0
        # varables for coverage
        all_rec_items = set()
        # varables for popularity
        popular_sum = 0
        predict_result=[]
        for i, user in enumerate(self.testset):
            if i % 500 == 0:
                print ('recommended for %d users' % i)
            rec_items = self.recommend(user,predict=True)
            predict_result.append((user,rec_items))
        print('Predict end...')
        return predict_result





In [3]:
test=pd.read_csv("H:\\pythonchengx_u\\Ecomme AI\\Date\\test_item")

train=pd.read_csv("H:\\pythonchengx_u\\Ecomme AI\\Date\\train_item")

In [6]:
itemcf = ItemBasedCF()

Similar item number = 100
Recommended item number = 30


In [28]:
itemcf.generate_dataset(train[['user_id','item_id']][:3000])#随机选择1W个交易记录测试

算法尝试！
split training set and test set succ
train set = 2130
test set = 870


In [38]:
itemcf.calc_item_sim()

counting items number and popularity...
count items number and popularity succ
total item number = 2049
building co-rated users matrix...
build co-rated users matrix succ
calculating item similarity matrix...
calculate item similarity matrix(similarity factor) succ
Total similarity factor number = 23314


In [39]:
itemcf.evaluate()

Evaluation start...
recommended for 0 users
precision=0.0001	recall=0.0012	coverage=0.2416	popularity=0.0876


In [7]:
itemcf.generate_dataset(train[['user_id','item_id']],test[['user_id','item_id']])


预测尝试！
split training set and test set succ
train set = 6010979
test set = 5252516


In [8]:
itemcf.calc_item_sim()

counting items number and popularity...
count items number and popularity succ
total item number = 1347477
building co-rated users matrix...
build co-rated users matrix succ
calculating item similarity matrix...
calculating item similarity factor(2000000)
calculating item similarity factor(4000000)
calculating item similarity factor(6000000)
calculating item similarity factor(8000000)
calculating item similarity factor(10000000)
calculating item similarity factor(12000000)
calculating item similarity factor(14000000)
calculating item similarity factor(16000000)
calculating item similarity factor(18000000)
calculating item similarity factor(20000000)
calculating item similarity factor(22000000)
calculating item similarity factor(24000000)
calculating item similarity factor(26000000)
calculating item similarity factor(28000000)
calculating item similarity factor(30000000)
calculating item similarity factor(32000000)
calculating item similarity factor(34000000)
calculating item similarity

In [9]:
import pickle
output = open('H:\\pythonchengx_u\\Ecomme AI\\code\\.pkl', 'wb')
pickle.dump(itemcf, output)
output.close()

In [11]:
result = itemcf.predict()



Predict start...
recommended for 0 users
recommended for 500 users
recommended for 1000 users
recommended for 1500 users
recommended for 2000 users
recommended for 2500 users
recommended for 3000 users
recommended for 3500 users
recommended for 4000 users
recommended for 4500 users
recommended for 5000 users
recommended for 5500 users
recommended for 6000 users
recommended for 6500 users
recommended for 7000 users
recommended for 7500 users
recommended for 8000 users
recommended for 8500 users
recommended for 9000 users
recommended for 9500 users
recommended for 10000 users
recommended for 10500 users
recommended for 11000 users
recommended for 11500 users
recommended for 12000 users
recommended for 12500 users
recommended for 13000 users
recommended for 13500 users
recommended for 14000 users
recommended for 14500 users
recommended for 15000 users
recommended for 15500 users
recommended for 16000 users
recommended for 16500 users
recommended for 17000 users
recommended for 17500 users

In [12]:
result_df = pd.DataFrame(result,columns=['buyer_admin_id','rec_items'])
result_df['rec_items'] = result_df.rec_items.apply(lambda x:[i[0] for i in x])



In [13]:
result_df.to_csv("H:\\pythonchengx_u\\Ecomme AI\\Date\\result_df",index=False)


In [14]:
result_df .head()


,buyer_admin_id,rec_items
0,1052625010,"[950243005, 1215389509, 1162718123, 717877431,..."
1,1067416116,"[883161172, 1007449376, 1163382798, 1091130606..."
2,1075634154,"[1125770956, 1244591772, 895989909, 1222325601..."
3,1013441031,"[1072016726, 1124994888, 1071900745, 100779201..."
4,129362515,"[1113013200, 857446973, 855681582, 1015624614,..."


In [16]:
itemcf.evaluate()


Evaluation start...
recommended for 0 users
recommended for 500 users
recommended for 1000 users
recommended for 1500 users
recommended for 2000 users
recommended for 2500 users
recommended for 3000 users
recommended for 3500 users
recommended for 4000 users
recommended for 4500 users
recommended for 5000 users
recommended for 5500 users
recommended for 6000 users
recommended for 6500 users
recommended for 7000 users
recommended for 7500 users
recommended for 8000 users
recommended for 8500 users
recommended for 9000 users
recommended for 9500 users
recommended for 10000 users
recommended for 10500 users
recommended for 11000 users
recommended for 11500 users
recommended for 12000 users
recommended for 12500 users
recommended for 13000 users
recommended for 13500 users
recommended for 14000 users
recommended for 14500 users
recommended for 15000 users
recommended for 15500 users
recommended for 16000 users
recommended for 16500 users
recommended for 17000 users
recommended for 17500 us